# Getting Started with LangChain Step by Step

Objective of this notebook:  
- Understand the **basic building blocks** of LangChain (LLM, prompt, chain).  
- Visualize a **simple pipeline** schema.  
- Build a mini **question/answer pipeline**, then a mini **simplified RAG pipeline**.

> Note: this notebook uses a **local LLM** (Ollama or LM Studio) configured via the `.env` file.

## Basic Building Blocks of LangChain

We will work with 3 main building blocks:

1. **LLM**: the language model (e.g., `gpt-4o-mini`)  
2. **Prompt**: how we talk to the model (system + user + variables)  
3. **Chain**: the **declarative** assembly of steps (processing pipeline)

We can imagine this very simple schema:

```text
              +---------------------------+
User input -> |   PromptTemplate          | -> formatted text
              +---------------------------+
                           |
                           v
                     +-----------+
                     |   LLM     | -> raw model response
                     +-----------+
                           |
                           v
                   +------------------+
                   | Output Parser    | -> string / JSON / etc.
                   +------------------+
```  
LangChain allows us to **describe this pipeline in Python**, rather than reprogramming everything manually.


## 🔧 Installation (do once in your environment)

Uncomment and execute the cell below if needed.


In [ ]:
# !pip install -U langchain-core langchain-ollama python-dotenv
# For LM Studio (OpenAI-compatible API):
# !pip install -U langchain-openai python-dotenv
# Optional: additional tools
# !pip install -U langchain-text-splitters

## ⚙️ LLM Initialization

We create a local LLM object which is the "language model" building block in LangChain.
You can use either **Ollama** or **LM Studio** by modifying the `.env` file.

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Local LLM configuration
llm_type = os.getenv("LOCAL_LLM_TYPE", "ollama")

if llm_type == "ollama":
    from langchain_ollama import ChatOllama
    
    ollama_base_url = os.getenv("OLLAMA_BASE_URL", "http://localhost:11434")
    ollama_model = os.getenv("OLLAMA_MODEL", "llama3.2:latest")
    
    model = ChatOllama(
        model=ollama_model,
        base_url=ollama_base_url,
        temperature=0.2,
    )
    print(f"Using Ollama with model {ollama_model}")
    
elif llm_type == "lmstudio":
    from langchain_openai import ChatOpenAI
    
    lmstudio_base_url = os.getenv("LMSTUDIO_BASE_URL", "http://localhost:1234/v1")
    lmstudio_model = os.getenv("LMSTUDIO_MODEL", "local-model")
    
    model = ChatOpenAI(
        model=lmstudio_model,
        base_url=lmstudio_base_url,
        api_key="not-needed",  # LM Studio doesn't require an API key
        temperature=0.2,
    )
    print(f"Using LM Studio with model {lmstudio_model}")
    
else:
    raise ValueError(f"Unrecognized LOCAL_LLM_TYPE: {llm_type}. Use 'ollama' or 'lmstudio'.")

model

## 1️⃣ First LLM Call via LangChain

We start with the simplest usage: we send a list of messages to the model.


In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are an educational assistant, clear and concise."),
    HumanMessage(content="Explain in two sentences what a language model is."),
]

response = model.invoke(messages)
print(response.content)


## 2️⃣ Separate the **prompt** and the **model**

Rather than building messages manually, we describe a **conversation template**.

We use `ChatPromptTemplate` to define:
- the system role (persona),
- the user message with a **variable `{question}`**.


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", f"You are an AI assistant who explains things for an audience of curious developers."),
        ("human", "Question: {question}"),
    ]
)


# LCEL: LangChain Expression Language
chain = prompt | model 

answer = chain.invoke({"question": "What is the difference between an LLM and LangChain?"})
print(answer)


In [ ]:
parser = StrOutputParser()

parser.invoke(answer)


👉 Here, we just instantiated the **previous schema**:

```text
variables -> PromptTemplate -> LLM -> OutputParser -> string
```

LangChain handles for us:
- message creation,
- model call,
- response conversion to `str`.


## 3️⃣ Add some "context" (mini-RAG without vector database)

We simulate here a **very simplified** RAG:  
- a mini "corpus" in memory,  
- a naive retrieval function (keyword search),  
- a prompt that combines **question + context**.

The goal is just to show the **"retrieval" building block** in the pipeline.


In [ ]:
# Mini "knowledge base" in memory
DOCUMENTS = [
    {
        "title": "Deterministic Agents",
        "content": "A deterministic agent follows a fixed scenario, without calling a language model at each step."
    },
    {
        "title": "AI Agents (LLM)",
        "content": "An AI agent uses a language model to dynamically decide actions at each step."
    },
    {
        "title": "LangChain",
        "content": "LangChain is a framework that helps structure LLM calls and integrate data, tools, and memory."
    },
]

def simple_retriever(query: str, k: int = 2) -> str:
    """
    Very simple:
    - calculates a score based on the number of common tokens
    - returns the k best documents concatenated
    """
    tokens = set(query.lower().split())
    scored = []
    for doc in DOCUMENTS:
        score = len(tokens.intersection(set(doc["content"].lower().split())))
        scored.append((score, doc))
    scored.sort(key=lambda x: x[0], reverse=True)
    top_docs = [doc["content"] for score, doc in scored[:k] if score > 0]
    if not top_docs:
        return "No relevant document found in the local database."
    return "\n\n---\n\n".join(top_docs)

print(simple_retriever("What is the difference between an AI agent and a deterministic agent?"))


### 🔗 Simplified RAG Chain 

We use `RunnableLambda` and `RunnablePassthrough` to build a pipeline:

```text
question
  ├──> (pass-through) ────────────────┐
  └──> retriever(question) -> context┤
                                      v
                           Prompt(question + context)
                                    |
                                    v
                                 LLM
                                    |
                                    v
                              OutputParser
```


In [ ]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# Step 1: prepare the "retriever" as Runnable
retriever_runnable = RunnableLambda(lambda q: simple_retriever(q))

# Step 2: define the prompt that integrates question + context
rag_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an assistant who answers by STRICTLY relying on the provided context.\n"
            "If information is not in the context, say so explicitly."
        ),
        (
            "human",
            "Context:\n{context}\n\n"
            "User question: {question}"
        ),
    ]
)

rag_chain = (
    {
        "question": RunnablePassthrough(),
        "context": retriever_runnable,
    }
    | rag_prompt
    | model
    | StrOutputParser()
)

question = "Explain the difference between a deterministic agent and an AI agent."
print(rag_chain.invoke(question))


## ✅ Recap
 
Building blocks presented in this notebook:
  - language model (`ChatOpenAI`),
  - prompts (`ChatPromptTemplate`),
  - pipeline (`chain = prompt | model | parser`),
  - retrieval/RAG (even very simplified).

Next we will cover:  
- **multi-step** orchestration with LangGraph,  
- then the more advanced **agent pattern** in the last notebook.
